Spotipy API

In [39]:
#Importing neccesary libraries and Cliend Cridentials Flow

import json
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import numpy as np
import os

In [40]:
#Creating Spotify App and getting ID´s

Client_ID = "c71bae50194549399a615967e53c6450"
Redirect_URI = "http://localhost:8080/callback/"


In [41]:
#Opening the secret file in read mode and assigning the json file to the variable "secret"

with open('Spotify_secret.json', "r") as f:
    secret = json.load(f)

In [42]:
#soptify app keys
sp_auth = spotipy.SpotifyOAuth(client_id = Client_ID,
                      client_secret = secret["secret"],
                      scope = 'user-read-recently-played,user-top-read',
                      redirect_uri = Redirect_URI)

In [43]:
#getting authorizatgion 
sp_auth.get_auth_response()

'AQDOwG738-TrlQKVzIuYkc4OBfHl1S6KNxYTMOWSTC5RYoUHuuCSN1KDsD8bqqUZni_pyquzZIsVuZINIdDt8RXrtpCW6pOwvcFIWxZj3rzb16NdKUbLj7pQQhVsNPE8R55epx553ASzkdmKV3h1GVtaeLQOxTpkcJuFmdw40wM-eivnVIrgNY17VvZyBXfZVtR0YjpH_oX2c5VSSojlE3KghmxOMWkF8AxzFOo'

In [44]:
#extra access via "current_user_recently_played()"
sp = spotipy.Spotify(auth_manager = sp_auth)

In [45]:
# creating dataset for top songs
my_top_songs = sp.current_user_top_tracks(limit = 20, time_range = "long_term")


dl_top = []
    
for item in my_top_songs["items"]:
    d = {}
    d["track_title"] = item["name"]
    da = []
    for artist in item["artists"]:
        da.append(artist["name"])
    d["artists"] = da
    d["popularity"] = item["popularity"]
    d["uri"] = item["uri"]
    d["duration"] = item["duration_ms"]
    dl_top.append(d)
my_top_songs_df = pd.DataFrame(dl_top)

#Save dataset for top songs
my_top_songs_df.to_csv("Data/Top_20_tracks.csv")

Dataset_1 = pd.read_csv("Data/Top_20_tracks.csv")

In [46]:
#Creating dataset for features
features = sp.audio_features(my_top_songs_df['uri'])

Df_features = pd.DataFrame(features)

#Save dataset for features
Df_features.to_csv("Data/Top_song_features.csv")

Dataset_2 = pd.read_csv("Data/Top_song_features.csv")

In [47]:
#Getting information about my top artists
my_top_artists = sp.current_user_top_artists(limit=20, offset=0, time_range='long_term')

# extracting the genre information for each artist and adding it to a list
genres_list = []
for artist in my_top_artists['items']:
    genres_list.extend(artist['genres'])

# count the frequency of each genre
genre_counts = {}
for genre in genres_list:
    if genre not in genre_counts:
        genre_counts[genre] = 1
    else:
        genre_counts[genre] += 1

# sort the genre counts and take the top 10
top_genres = sorted(genre_counts.items(), key=lambda x: x[1], reverse=True)[:10]

# create a dataframe from the top genres data
df = pd.DataFrame(top_genres, columns=['genre', 'count'])

# save dataset for top genres
df.to_csv("Data/Top_genres.csv", index=False)